#### <mark>**IMPORT PY LIBS**</mark>

In [22]:
import csv, io, json, urllib.request, urllib.parse, urllib.error, base64, math, re
from typing import Optional, Union
from datetime import date, datetime as dt, timedelta as td

#### <mark>**EMAILER ENGINE**</mark>

In [ ]:

class Emailer:

    def __init__(self, tenant: str, client_id: str, client_secret: str, sender_upn: str):
        self.TENANT = tenant
        self.CLIENT_ID = client_id
        self.CLIENT_SECRET = client_secret
        self.SENDER_UPN = sender_upn

    _tok = lambda self: json.loads(
        urllib.request.urlopen(
            urllib.request.Request(
                f"https://login.microsoftonline.com/{self.TENANT}/oauth2/v2.0/token",
                data=urllib.parse.urlencode({
                    "client_id": self.CLIENT_ID,
                    "client_secret": self.CLIENT_SECRET,
                    "scope": "https://graph.microsoft.com/.default",
                    "grant_type": "client_credentials"
                }).encode(),
                headers={"Content-Type":"application/x-www-form-urlencoded"}
            )
        ).read()
    )["access_token"]

    # ---------- private helpers ----------
    def _recips(self, to_list):
        return [{"emailAddress": {"address": a}} for a in to_list]

    def _post(self, message_dict):
        body = json.dumps({
            "message": message_dict,
            "saveToSentItems": True  # flip to False if testing and Sent Items policy is noisy
        }).encode()

        req = urllib.request.Request(
            f"https://graph.microsoft.com/v1.0/users/{self.SENDER_UPN}/sendMail",
            data=body,
            headers={
                "Authorization": f"Bearer {self._tok()}",
                "Content-Type": "application/json"
            }
        )
        try:
            with urllib.request.urlopen(req) as r:
                return r.getcode() in (202, 200, 204)
        except urllib.error.HTTPError as e:
            print("HTTPError", e.code, e.reason)
            try:
                print("Graph body:", e.read().decode())
            except:
                pass
            raise

    # ---------- public API ----------
    def send_text(self, to_list, object, subject="Fabric Notebook"):
        """
        Send plain text. 'object' must be a string.
        """
        msg = {
            "subject": subject,
            "body": {"contentType": "Text", "content": str(object)},
            "toRecipients": self._recips(to_list)
        }
        return self._post(msg)

    def send_html(self, to_list, object, subject="Fabric Notebook"):
        """
        Send HTML body. 'object' is a (possibly multi-line) HTML string.
        """
        msg = {
            "subject": subject,
            "body": {"contentType": "HTML", "content": str(object)},
            "toRecipients": self._recips(to_list)
        }
        return self._post(msg)

    def send_csv(self, to_list, object, subject="Fabric Notebook"):
        """
        Send a CSV as attachment.
        'object' can be:
          - str  : CSV text
          - bytes: raw CSV bytes
          - pandas.DataFrame: will be converted to CSV (index=False)
        """
        # Convert object -> bytes
        csv_bytes = None
        # pandas DataFrame?
        if hasattr(object, "to_csv"):
            try:
                csv_text = object.to_csv(index=False)
                csv_bytes = csv_text.encode("utf-8")
            except Exception:
                # fallback to str()
                csv_bytes = str(object).encode("utf-8")
        elif isinstance(object, bytes):
            csv_bytes = object
        else:
            csv_bytes = str(object).encode("utf-8")

        attachment = {
            "@odata.type": "#microsoft.graph.fileAttachment",
            "name": "data.csv",
            "contentType": "text/csv",
            "contentBytes": base64.b64encode(csv_bytes).decode("utf-8")
        }

        msg = {
            "subject": subject,
            "body": {"contentType": "Text", "content": "Please find the CSV attached."},
            "toRecipients": self._recips(to_list),
            "attachments": [attachment]
        }
        return self._post(msg)

# ==================================================== INIT ENGINE =======================================================

mailer = Emailer(tenant="0000"
    ,client_id="0000"
    ,client_secret="0000"
    ,sender_upn="pbi.notification@iss-gf.com")

#### **<mark>QUERY ENGINE</mark>**

In [24]:

class Engine:
    """
    FabricDax(model, workspace=None)
      .dax("EVALUATE ...", csv=None) -> scalar (1x1) or CSV string (else)
      Set csv=True/False to force output; None = auto.
    """
    def __init__(self, model: str, workspace: Optional[str] = None):
        self.model, self.workspace = model, workspace

    def dax(self, query: str, *, csv: Optional[bool] = None) -> Union[str, int, float]:
        df = None
        try:
            from sempy.fabric import evaluate_dax
            if self.workspace:
                from sempy.fabric import set_workspace
                set_workspace(self.workspace)
            df = evaluate_dax(self.model, query)
        except Exception as e1:
            try:
                # Fallback for environments with sempy-labs helpers available
                from sempy_labs.dax import evaluate_dax as labs_evaluate_dax
                df = labs_evaluate_dax(self.model, query)  # same idea: returns a DataFrame
            except Exception as e2:
                raise RuntimeError(f"Unable to run DAX via sempy or sempy-labs: {e1} / {e2}")

        if df.shape == (1, 1) and (csv is None or csv is False):
            val = df.iat[0, 0]
            return val if not isinstance(val, (list, dict)) else str(val)
        return df.to_csv(index=False)

# =================================================== INIT ENGINE ========================================================

query = Engine(model="c8ccb1b4-1c84-4fac-b5fe-5fc677cf6d34")


#### **<mark>DATE SETTERS</mark>**

In [25]:


def get_target_date(date_str=None):
    """
    Returns last date of the prev month if the provided date's day is <= 10.
    If no date provided, uses current date and applies the same "IF" logic.
    """
    # Determine the base date
    if date_str:
        # Convert string "YYYY-MM-DD" to a date object
        current_date = dt.strptime(date_str, '%Y-%m-%d').date()
    else:
        current_date = date.today()
    
    # Check logic: <=10
    if current_date.day <= 10:
        # Get first day of current month, then subtract 1 day to get last of prev month
        target_date = current_date.replace(day=1) - td(days=1)
    else:
        target_date = current_date- td(days=1)
        
    return target_date.strftime('%Y-%m-%d')

def dax_date_filter(date_dict: dict) -> dict:
    """
    Transforms ISO date strings 'YYYY-MM-DD' into 'DATE(YYYY,M,D)' format.
    Removes zero-padding from Month and Day.
    """
    return {
        key: f"DATE({int(val[:4])},{int(val[5:7])},{int(val[8:])})"
        for key, val in date_dict.items()
    }

def dates(now = None):
    """
        !DOC!
        arg 'NOW' format YYYY-MM-DD
    """
    today = date.today() if now == None else dt.strptime(now,'%Y-%m-%d')
    now = date.today() - td(days=1) if now == None else dt.strptime(now,'%Y-%m-%d')
    y, m, d = now.year, now.month, now.day

    fmt = lambda dt: dt.strftime('%Y-%m-%d')

    start_ytd = date(y, 1, 1)
    start_mtd = date(y, m, 1)

    py = y - 1

    return {
        'MTD_START': fmt(start_mtd),
        'MTD_END': fmt(now),
        'YTD_START': fmt(start_ytd),
        'YTD_END': fmt(now),
        'MTD_START_PY': fmt(date(py, m, 1)),
        'MTD_END_PY': fmt( (today.replace(year=today.year-1, day=28) + td(days=4)).replace(day=1) - td(days=1) ),
        'YTD_START_PY': fmt(date(py, 1, 1)),
        'YTD_END_PY': fmt( (today.replace(year=today.year-1, day=28) + td(days=4)).replace(day=1) - td(days=1) ),
        'MTD_START_PY_FM': fmt(date(py, m, 1)),
        'MTD_END_PY_FM': fmt( (today.replace(year=today.year-1, day=28) + td(days=4)).replace(day=1) - td(days=1) ),
        'YTD_START_PY_FM': fmt(date(py, 1, 1)),
        'YTD_END_PY_FM': fmt( (today.replace(year=today.year-1, day=28) + td(days=4)).replace(day=1) - td(days=1) )
    }

_dates = dates(get_target_date())
_filters = dax_date_filter(_dates)


#### **<mark>MAIN METRICS</mark>**

In [26]:
# ------------------------------------------------------------------------------------------------------------------------ REV MTD 

rev_mtd = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START
                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END
                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('MTD_START', _filters['MTD_START']).replace('MTD_END', _filters['MTD_END']))

# ------------------------------------------------------------------------------------------------------------------------ GP MTD

gp_mtd = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START
                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END
                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('MTD_START', _filters['MTD_START']).replace('MTD_END', _filters['MTD_END']))






# ------------------------------------------------------------------------------------------------------------------------ REV YTD


rev_ytd = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= YTD_START
                            ,'DAX'[ACCOUNTINGDATE] <= YTD_END
                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('YTD_START', _filters['YTD_START']).replace('YTD_END', _filters['YTD_END']))

# -------------------------------------------------------------------------------------- GP YTD (YEAR TO DATE) ----------------------------------------


gp_ytd = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= YTD_START
                            ,'DAX'[ACCOUNTINGDATE] <= YTD_END
                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('YTD_START', _filters['YTD_START']).replace('YTD_END', _filters['YTD_END']))


# -------------------------------------------------------- MARGIN METRICS

gp_margin_mtd = gp_mtd / rev_mtd

gp_margin_ytd = gp_ytd / rev_ytd




chw_mtd = query.dax("""
                    EVALUATE
                        {CALCULATE(
                            SUM('Shipments PBI'[Chargeable Weight (Air)])
                                ,'Shipments PBI'[Contact Type] IN {"client","agent"}
                                ,'Shipments PBI'[Operational Date] >= MTD_START
                                ,'Shipments PBI'[Operational Date] <= MTD_END
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                        )}
""".replace('MTD_START', _filters['MTD_START']).replace('MTD_END', _filters['MTD_END']))

chw_ytd = query.dax("""
                    EVALUATE
                        {CALCULATE(
                            SUM('Shipments PBI'[Chargeable Weight (Air)])
                                ,'Shipments PBI'[Contact Type] IN {"client","agent"}
                                ,'Shipments PBI'[Operational Date] >= YTD_START
                                ,'Shipments PBI'[Operational Date] <= YTD_END
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                        )}
""".replace('YTD_START', _filters['YTD_START']).replace('YTD_END', _filters['YTD_END']))

# ------------------------------------------------------------------------------------------------------------------------ TEUs MTD

#OK
teu_mtd = query.dax("""
                    EVALUATE
                        {CALCULATE(
                            SUM('Shipments PBI'[TEU (FCL)])
                                ,'Shipments PBI'[Contact Type] IN {"client","agent"}
                                ,'Shipments PBI'[Operational Date] >= MTD_START
                                ,'Shipments PBI'[Operational Date] <= MTD_END
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                        )}
""".replace('MTD_START', _filters['MTD_START']).replace('MTD_END', _filters['MTD_END']))


# ------------------------------------------------------------------------------------------------------------------------ TEUs YTD

teu_ytd = query.dax("""
                    EVALUATE
                        {CALCULATE(
                            SUM('Shipments PBI'[TEU (FCL)])
                                ,'Shipments PBI'[Contact Type] IN {"client","agent"}
                                ,'Shipments PBI'[Operational Date] >= YTD_START
                                ,'Shipments PBI'[Operational Date] <= YTD_END
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                        )}
""".replace('YTD_START', _filters['YTD_START']).replace('YTD_END', _filters['YTD_END']))





#### <mark>**PY MAIN METRICS**</mark>

In [27]:

# ----------------------------------------------------------------- MTD

rev_mtd_py_base = query.dax("""
				EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START_PY
                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END_PY
                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ","SHQ","Sierra Leone"
                                                                        ,"Sri Lanka","Taiwan","Tanzania"
                                                                        ,"Uganda","UK"
                                                                        })
                                                            )}
""".replace('MTD_START_PY', _filters['MTD_START_PY']).replace('MTD_END_PY', _filters['MTD_END_PY']))

rev_mtd_py = rev_mtd_py_base if dt.now().day <= 10 else (rev_mtd_py_base / ((date.today().replace(day=28) + td(days=4)).replace(day=1) - td(days=1)).day * (dt.now().day - 1)   )


gp_mtd_py_base = query.dax("""
				EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START_PY
                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END_PY
                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ","SHQ","Sierra Leone"
                                                                        ,"Sri Lanka","Taiwan","Tanzania"
                                                                        ,"Uganda","UK"
                                                                        })
                                                            )}
""".replace('MTD_START_PY', _filters['MTD_START_PY']).replace('MTD_END_PY', _filters['MTD_END_PY'])) 

gp_mtd_py = gp_mtd_py_base if dt.now().day <= 10 else (gp_mtd_py_base / ((date.today().replace(day=28) + td(days=4)).replace(day=1) - td(days=1)).day * (dt.now().day - 1)   )



# --------------------------------------------------------------- YTD 



rev_ytd_py_base = query.dax("""
				EVALUATE
                    VAR _RefDate = TODAY()
                    VAR _Day = DAY( _RefDate )
                    VAR _Month = MONTH( _RefDate )
                    VAR _Year = YEAR( _RefDate )

                    -- Logic Conditions
                    VAR _IsJanEarly = _Month = 1 && _Day <= 10
                    VAR _IsJanLate = _Month = 1 && _Day > 10
                    -- Note: Conditions 3 and 4 (Month > 1) are handled in the SWITCH default or specific branches below

                    -- 1. Determine Start Date
                    -- Logic: Only "Jan Early" goes back 2 years. All other scenarios start at Jan 1 of Prev Year.
                    VAR _StartDate = 
                        IF( 
                            _IsJanEarly, 
                            DATE( _Year - 2, 1, 1 ), 
                            DATE( _Year - 1, 1, 1 ) 
                        )

                    -- 2. Determine End Date
                    VAR _EndDate = 
                        SWITCH( TRUE(),
                            
                            -- Cond 1: Jan Early => Last date of the current year set 2 years back (Dec 31, Year-2)
                            _IsJanEarly, DATE( _Year - 2, 12, 31 ),

                            -- Cond 2: Jan Late => Last date of January set one year back (Jan 31, Year-1)
                            _IsJanLate, DATE( _Year - 1, 1, 31 ),

                            -- Cond 3 & 4: Month > 1 => Last day of prev month set one year back
                            -- (Logic is identical for Day <= 10 and Day > 10 based on your requirement)
                            -- Calculates the last day of the previous month relative to the reference date, shifted 1 year back.
                            EDATE( EOMONTH( _RefDate, -1 ), -12 )
                        )
                    RETURN
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= _StartDate
                            ,'DAX'[ACCOUNTINGDATE] <= _EndDate
                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ","SHQ","Sierra Leone"
                                                                        ,"Sri Lanka","Taiwan","Tanzania"
                                                                        ,"Uganda","UK"
                                                                        })
                                                            )}
""")

rev_ytd_py = rev_ytd_py_base if dt.now().day <= 10 else ( rev_ytd_py_base + rev_mtd_py )


gp_ytd_py_base = query.dax("""
				EVALUATE
                    VAR _RefDate = TODAY()
                    VAR _Day = DAY( _RefDate )
                    VAR _Month = MONTH( _RefDate )
                    VAR _Year = YEAR( _RefDate )

                    -- Logic Conditions
                    VAR _IsJanEarly = _Month = 1 && _Day <= 10
                    VAR _IsJanLate = _Month = 1 && _Day > 10
                    -- Note: Conditions 3 and 4 (Month > 1) are handled in the SWITCH default or specific branches below

                    -- 1. Determine Start Date
                    -- Logic: Only "Jan Early" goes back 2 years. All other scenarios start at Jan 1 of Prev Year.
                    VAR _StartDate = 
                        IF( 
                            _IsJanEarly, 
                            DATE( _Year - 2, 1, 1 ), 
                            DATE( _Year - 1, 1, 1 ) 
                        )

                    -- 2. Determine End Date
                    VAR _EndDate = 
                        SWITCH( TRUE(),
                            
                            -- Cond 1: Jan Early => Last date of the current year set 2 years back (Dec 31, Year-2)
                            _IsJanEarly, DATE( _Year - 2, 12, 31 ),

                            -- Cond 2: Jan Late => Last date of January set one year back (Jan 31, Year-1)
                            _IsJanLate, DATE( _Year - 1, 1, 31 ),

                            -- Cond 3 & 4: Month > 1 => Last day of prev month set one year back
                            -- (Logic is identical for Day <= 10 and Day > 10 based on your requirement)
                            -- Calculates the last day of the previous month relative to the reference date, shifted 1 year back.
                            EDATE( EOMONTH( _RefDate, -1 ), -12 )
                        )
                    RETURN
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= _StartDate
                            ,'DAX'[ACCOUNTINGDATE] <= _EndDate
                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ","SHQ","Sierra Leone"
                                                                        ,"Sri Lanka","Taiwan","Tanzania"
                                                                        ,"Uganda","UK"
                                                                        })
                                                            )}
""")

gp_ytd_py = gp_ytd_py_base if dt.now().day <= 10 else ( gp_ytd_py_base + gp_mtd_py)


# ------------------------------------------------------------------ MARGIN METRICS

gp_margin_mtd_py = gp_mtd_py / rev_mtd_py

gp_margin_ytd_py = gp_ytd_py / rev_ytd_py


# ---------------------------------------------------------------------- CHW / TEU


chw_mtd_py = query.dax("""
                    EVALUATE
                    VAR _RefDate = TODAY()
                    VAR _Day = DAY( _RefDate )

                    -- Logic Flag
                    VAR _IsEarly = _Day <= 10

                    -- 1. Determine Start Date
                    VAR _StartDate = 
                        IF( 
                            _IsEarly,
                            -- Early (Day <= 10): Start of Prev Month set 1 year back
                            -- EOMONTH(..., -2) + 1 gives the first day of the previous month
                            EDATE( EOMONTH( _RefDate, -2 ) + 1, -12 ),

                            -- Late (Day > 10): Start of Current Month set 1 year back
                            -- EOMONTH(..., -1) + 1 gives the first day of the current month
                            EDATE( EOMONTH( _RefDate, -1 ) + 1, -12 )
                        )

                    -- 2. Determine End Date
                    VAR _EndDate = 
                        IF( 
                            _IsEarly,
                            -- Early (Day <= 10): End of Prev Month set 1 year back
                            EDATE( EOMONTH( _RefDate, -1 ), -12 ),

                            -- Late (Day > 10): Current Date - 1 Day, set 1 year back
                            EDATE( _RefDate - 1, -12 )
                        )
                    RETURN
                        {CALCULATE(
                            SUM('Shipments PBI'[Chargeable Weight (Air)])
                                ,'Shipments PBI'[Contact Type] IN {"client","agent"}
                                ,'Shipments PBI'[Operational Date] >= _StartDate
                                ,'Shipments PBI'[Operational Date] <= _EndDate
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                        )}
""")



# ------------------------------------------------------------------------------------------------------------------------ CHWs YTD



chw_ytd_py = query.dax("""
                    EVALUATE
                    VAR _RefDate = TODAY()
                    VAR _Day = DAY( _RefDate )
                    VAR _Month = MONTH( _RefDate )
                    VAR _Year = YEAR( _RefDate )

                    -- Logic Flags
                    VAR _IsJanEarly = _Month = 1 && _Day <= 10
                    VAR _IsOtherEarly = _Month > 1 && _Day <= 10

                    -- 1. Determine Start Date
                    -- Logic: Only "Jan Early" goes back 2 years. All other scenarios start at Jan 1 of Prev Year.
                    VAR _StartDate = 
                        IF( 
                            _IsJanEarly, 
                            DATE( _Year - 2, 1, 1 ), 
                            DATE( _Year - 1, 1, 1 ) 
                        )

                    -- 2. Determine End Date
                    VAR _EndDate = 
                        SWITCH( TRUE(),
                            
                            -- Scenario 1: Jan <= 10 
                            -- Logic: Last date of current year set 2 years back (Dec 31, Year-2)
                            _IsJanEarly, DATE( _Year - 2, 12, 31 ),

                            -- Scenario 3: Month > 1 AND Date <= 10
                            -- Logic: Last date of the prev month (relative to current) set one year back
                            _IsOtherEarly, EDATE( EOMONTH( _RefDate, -1 ), -12 ),

                            -- Scenario 2 (Jan > 10) & Scenario 4 (Month > 1 AND Date > 10)
                            -- Logic: Current date minus one day set one year back.
                            -- Note: This catch-all covers both scenarios as the math is identical.
                            EDATE( _RefDate - 1, -12 )
                        )
                    RETURN
                        {CALCULATE(
                            SUM('Shipments PBI'[Chargeable Weight (Air)])
                                ,'Shipments PBI'[Contact Type] IN {"client","agent"}
                                ,'Shipments PBI'[Operational Date] >= _StartDate
                                ,'Shipments PBI'[Operational Date] <= _EndDate
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                        )}
""")


# ------------------------------------------------------------------------------------------------------------------------ TEU YTD


teu_mtd_py = query.dax("""
                    EVALUATE
                    VAR _RefDate = TODAY()
                    VAR _Day = DAY( _RefDate )

                    -- Logic Flag
                    VAR _IsEarly = _Day <= 10

                    -- 1. Determine Start Date
                    VAR _StartDate = 
                        IF( 
                            _IsEarly,
                            -- Early (Day <= 10): Start of Prev Month set 1 year back
                            -- EOMONTH(..., -2) + 1 gives the first day of the previous month
                            EDATE( EOMONTH( _RefDate, -2 ) + 1, -12 ),

                            -- Late (Day > 10): Start of Current Month set 1 year back
                            -- EOMONTH(..., -1) + 1 gives the first day of the current month
                            EDATE( EOMONTH( _RefDate, -1 ) + 1, -12 )
                        )

                    -- 2. Determine End Date
                    VAR _EndDate = 
                        IF( 
                            _IsEarly,
                            -- Early (Day <= 10): End of Prev Month set 1 year back
                            EDATE( EOMONTH( _RefDate, -1 ), -12 ),

                            -- Late (Day > 10): Current Date - 1 Day, set 1 year back
                            EDATE( _RefDate - 1, -12 )
                        )
                    RETURN
                        {CALCULATE(
                            SUM('Shipments PBI'[TEU (FCL)])
                                ,'Shipments PBI'[Contact Type] IN {"client","agent"}
                                ,'Shipments PBI'[Operational Date] >= _StartDate
                                ,'Shipments PBI'[Operational Date] <= _EndDate
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                        )}
""")


teu_ytd_py = query.dax("""
                    EVALUATE
                    VAR _RefDate = TODAY()
                    VAR _Day = DAY( _RefDate )
                    VAR _Month = MONTH( _RefDate )
                    VAR _Year = YEAR( _RefDate )

                    -- Logic Flags
                    VAR _IsJanEarly = _Month = 1 && _Day <= 10
                    VAR _IsOtherEarly = _Month > 1 && _Day <= 10

                    -- Start Date
                    -- Logic: Only "Jan Early" goes back 2 years. All other scenarios start at Jan 1 of Prev Year.
                    VAR _StartDate = 
                        IF( 
                            _IsJanEarly, 
                            DATE( _Year - 2, 1, 1 ), 
                            DATE( _Year - 1, 1, 1 ) 
                        )

                    -- End Date
                    VAR _EndDate = 
                        SWITCH( TRUE(),
                            
                            -- Scenario 1: Jan <= 10 
                            -- Logic: Last date of current year set 2 years back (Dec 31, Year-2)
                            _IsJanEarly, DATE( _Year - 2, 12, 31 ),

                            -- Scenario 3: Month > 1 AND Date <= 10
                            -- Logic: Last date of the prev month (relative to current) set one year back
                            _IsOtherEarly, EDATE( EOMONTH( _RefDate, -1 ), -12 ),

                            -- Scenario 2 (Jan > 10) & Scenario 4 (Month > 1 AND Date > 10)
                            -- Logic: Current date minus one day set one year back.
                            -- Note: This catch-all covers both scenarios as the math is identical.
                            EDATE( _RefDate - 1, -12 )
                        )
                    RETURN
                        {CALCULATE(
                            SUM('Shipments PBI'[TEU (FCL)])
                                ,'Shipments PBI'[Contact Type] IN {"client","agent"}
                                ,'Shipments PBI'[Operational Date] >= _StartDate
                                ,'Shipments PBI'[Operational Date] <= _EndDate
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                        )}
""")

#### <mark>**CL METRICS**</mark>

In [28]:

rev_mtd_cl = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START
                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END
                            ,'DAX'[LINEOFBUSINESS] = "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('MTD_START', _filters['MTD_START']).replace('MTD_END', _filters['MTD_END']))



gp_mtd_cl = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START
                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END
                            ,'DAX'[LINEOFBUSINESS] = "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('MTD_START', _filters['MTD_START']).replace('MTD_END', _filters['MTD_END']))




rev_ytd_cl = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= YTD_START
                            ,'DAX'[ACCOUNTINGDATE] <= YTD_END
                            ,'DAX'[LINEOFBUSINESS] = "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('YTD_START', _filters['YTD_START']).replace('YTD_END', _filters['YTD_END']))

#OK
gp_ytd_cl = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= YTD_START
                            ,'DAX'[ACCOUNTINGDATE] <= YTD_END
                            ,'DAX'[LINEOFBUSINESS] = "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('YTD_START', _filters['YTD_START']).replace('YTD_END', _filters['YTD_END']))



############################################################################## PY ##############################################################################


rev_mtd_cl_py_base = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START_PY
                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END_PY
                            ,'DAX'[LINEOFBUSINESS] = "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('MTD_START_PY', _filters['MTD_START_PY']).replace('MTD_END_PY', _filters['MTD_END_PY']))

rev_mtd_cl_py = rev_mtd_cl_py_base if dt.now().day <= 10 else ( rev_mtd_cl_py_base / ((date.today().replace(day=28) + td(days=4)).replace(day=1) - td(days=1)).day * (dt.now().day - 1) )




gp_mtd_cl_py_base = query.dax("""
                    EVALUATE
                        {CALCULATE(
                            SUM('DAX'[AMOUNT_TAGETIK])
                                ,ALL('DAX'[ACCOUNTINGDATE])
                                ,'DAX'[ACCOUNTINGDATE] >= MTD_START_PY
                                ,'DAX'[ACCOUNTINGDATE] <= MTD_END_PY
                                ,'DAX'[LINEOFBUSINESS] = "CNLS"
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ","SHQ","Sierra Leone"
                                                                            ,"Sri Lanka","Taiwan","Tanzania"
                                                                            ,"Uganda","UK"
                                                                            })
                                                                )}
""".replace('MTD_START_PY', _filters['MTD_START_PY']).replace('MTD_END_PY', _filters['MTD_END_PY']))

gp_mtd_cl_py = gp_mtd_cl_py_base if dt.now().day <= 10 else ( gp_mtd_cl_py_base / ((date.today().replace(day=28) + td(days=4)).replace(day=1) - td(days=1)).day * (dt.now().day - 1) )



rev_ytd_cl_py_base = query.dax("""
                    EVALUATE
                        VAR _RefDate = TODAY()
                        VAR _Day = DAY( _RefDate )
                        VAR _Month = MONTH( _RefDate )
                        VAR _Year = YEAR( _RefDate )

                        -- Logic Conditions
                        VAR _IsJanEarly = _Month = 1 && _Day <= 10
                        VAR _IsJanLate = _Month = 1 && _Day > 10
                        -- Note: Conditions 3 and 4 (Month > 1) are handled in the SWITCH default or specific branches below

                        -- 1. Determine Start Date
                        -- Logic: Only "Jan Early" goes back 2 years. All other scenarios start at Jan 1 of Prev Year.
                        VAR _StartDate = 
                            IF( 
                                _IsJanEarly, 
                                DATE( _Year - 2, 1, 1 ), 
                                DATE( _Year - 1, 1, 1 ) 
                            )

                        -- 2. Determine End Date
                        VAR _EndDate = 
                            SWITCH( TRUE(),
                                
                                -- Cond 1: Jan Early => Last date of the current year set 2 years back (Dec 31, Year-2)
                                _IsJanEarly, DATE( _Year - 2, 12, 31 ),

                                -- Cond 2: Jan Late => Last date of January set one year back (Jan 31, Year-1)
                                _IsJanLate, DATE( _Year - 1, 1, 31 ),

                                -- Cond 3 & 4: Month > 1 => Last day of prev month set one year back
                                -- (Logic is identical for Day <= 10 and Day > 10 based on your requirement)
                                -- Calculates the last day of the previous month relative to the reference date, shifted 1 year back.
                                EDATE( EOMONTH( _RefDate, -1 ), -12 )
                            )
                        RETURN
                        {CALCULATE(
                            SUM('DAX'[AMOUNT_TAGETIK])
                                ,'DAX'[Name] = "Revenue"
                                ,ALL('DAX'[ACCOUNTINGDATE])
                                ,'DAX'[ACCOUNTINGDATE] >= _StartDate
                                ,'DAX'[ACCOUNTINGDATE] <= _EndDate
                                ,'DAX'[LINEOFBUSINESS] = "CNLS"
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ", "SHQ", "Sierra Leone"
                                                                            ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                            ,"Uganda", "UK"
                                                                            })
                                                                )}
""")

rev_ytd_cl_py = rev_ytd_cl_py_base if dt.now().day <= 10 else ( rev_ytd_cl_py_base + rev_mtd_cl_py )




gp_ytd_cl_py_base = query.dax("""
                    EVALUATE
                        VAR _RefDate = TODAY()
                        VAR _Day = DAY( _RefDate )
                        VAR _Month = MONTH( _RefDate )
                        VAR _Year = YEAR( _RefDate )

                        -- Logic Conditions
                        VAR _IsJanEarly = _Month = 1 && _Day <= 10
                        VAR _IsJanLate = _Month = 1 && _Day > 10
                        -- Note: Conditions 3 and 4 (Month > 1) are handled in the SWITCH default or specific branches below

                        -- 1. Determine Start Date
                        -- Logic: Only "Jan Early" goes back 2 years. All other scenarios start at Jan 1 of Prev Year.
                        VAR _StartDate = 
                            IF( 
                                _IsJanEarly, 
                                DATE( _Year - 2, 1, 1 ), 
                                DATE( _Year - 1, 1, 1 ) 
                            )

                        -- 2. Determine End Date
                        VAR _EndDate = 
                            SWITCH( TRUE(),
                                
                                -- Cond 1: Jan Early => Last date of the current year set 2 years back (Dec 31, Year-2)
                                _IsJanEarly, DATE( _Year - 2, 12, 31 ),

                                -- Cond 2: Jan Late => Last date of January set one year back (Jan 31, Year-1)
                                _IsJanLate, DATE( _Year - 1, 1, 31 ),

                                -- Cond 3 & 4: Month > 1 => Last day of prev month set one year back
                                -- (Logic is identical for Day <= 10 and Day > 10 based on your requirement)
                                -- Calculates the last day of the previous month relative to the reference date, shifted 1 year back.
                                EDATE( EOMONTH( _RefDate, -1 ), -12 )
                            )
                        RETURN
                        {CALCULATE(
                            SUM('DAX'[AMOUNT_TAGETIK])
                                ,ALL('DAX'[ACCOUNTINGDATE])
                                ,'DAX'[ACCOUNTINGDATE] >= _StartDate
                                ,'DAX'[ACCOUNTINGDATE] <= _EndDate
                                ,'DAX'[LINEOFBUSINESS] = "CNLS"
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ","SHQ","Sierra Leone"
                                                                            ,"Sri Lanka","Taiwan","Tanzania"
                                                                            ,"Uganda","UK"
                                                                            })
                                                                )}
""")

gp_ytd_cl_py = gp_ytd_cl_py_base if dt.now().day <= 10 else ( gp_ytd_cl_py_base + gp_mtd_cl_py )




#### <mark>**CL METRICS FM (FULL MONTH)**</mark>

In [29]:
rev_mtd_py_fm = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START_PY_FM
                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END_PY_FM
                            ,'DAX'[LINEOFBUSINESS] = "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('MTD_START_PY_FM', _filters['MTD_START_PY_FM']).replace('MTD_END_PY_FM', _filters['MTD_END_PY_FM']))

gp_mtd_py_fm = query.dax("""
                    EVALUATE
                        {CALCULATE(
                            SUM('DAX'[AMOUNT_TAGETIK])
                                ,ALL('DAX'[ACCOUNTINGDATE])
                                ,'DAX'[ACCOUNTINGDATE] >= MTD_START_PY_FM
                                ,'DAX'[ACCOUNTINGDATE] <= MTD_END_PY_FM
                                ,'DAX'[LINEOFBUSINESS] = "CNLS"
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ","SHQ","Sierra Leone"
                                                                            ,"Sri Lanka","Taiwan","Tanzania"
                                                                            ,"Uganda","UK"
                                                                            })
                                                                )}
""".replace('MTD_START_PY_FM', _filters['MTD_START_PY_FM']).replace('MTD_END_PY_FM', _filters['MTD_END_PY_FM']))

# ---------------------------------- FULL MONTH (YTD)

rev_ytd_py_fm = query.dax("""
                    EVALUATE
                    {CALCULATE(
                        SUM('DAX'[AMOUNT_TAGETIK])
                            ,'DAX'[Name] = "Revenue"
                            ,ALL('DAX'[ACCOUNTINGDATE])
                            ,'DAX'[ACCOUNTINGDATE] >= YTD_START_PY_FM
                            ,'DAX'[ACCOUNTINGDATE] <= YTD_END_PY_FM
                            ,'DAX'[LINEOFBUSINESS] = "CNLS"
                            ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                        ,"Congo","Djibouti","Germany",
                                                                        "GHQ", "Kenya", "Korea",
                                                                        "Luxembourg", "Mozambique", "Netherlands",
                                                                        "SAHQ", "SHQ", "Sierra Leone"
                                                                        ,"Sri Lanka", "Taiwan", "Tanzania"
                                                                        ,"Uganda", "UK"
                                                                        })
                                                            )}
""".replace('YTD_START_PY_FM', _filters['YTD_START_PY_FM']).replace('YTD_END_PY_FM', _filters['YTD_END_PY_FM']))

gp_ytd_py_fm = query.dax("""
                    EVALUATE
                        {CALCULATE(
                            SUM('DAX'[AMOUNT_TAGETIK])
                                ,ALL('DAX'[ACCOUNTINGDATE])
                                ,'DAX'[ACCOUNTINGDATE] >= YTD_START_PY_FM
                                ,'DAX'[ACCOUNTINGDATE] <= YTD_END_PY_FM
                                ,'DAX'[LINEOFBUSINESS] = "CNLS"
                                ,NOT('DAX Region Catalog'[Country Name] in {"Côte d'Ivoire"
                                                                            ,"Congo","Djibouti","Germany",
                                                                            "GHQ", "Kenya", "Korea",
                                                                            "Luxembourg", "Mozambique", "Netherlands",
                                                                            "SAHQ","SHQ","Sierra Leone"
                                                                            ,"Sri Lanka","Taiwan","Tanzania"
                                                                            ,"Uganda","UK"
                                                                            })
                                                                )}
""".replace('YTD_START_PY_FM', _filters['YTD_START_PY_FM']).replace('YTD_END_PY_FM', _filters['YTD_END_PY_FM']))

#### <mark>**BUILD METRICS OBJ**</mark>

In [30]:
metrics = [
    {
        'name': 'Revenue'
        ,'change': f"{((rev_ytd - rev_ytd_py) / rev_ytd_py):.1%}"
        ,'mtd':  f"{rev_mtd:,.0f}"
        ,'mtd_py':  f"{rev_mtd_py:,.0f}"
        ,'ytd':  f"{rev_ytd:,.0f}"
        ,'ytd_py':  f"{rev_ytd_py:,.0f}"
    },
    {
        'name': 'GP'
        ,'change': f"{((gp_ytd - gp_ytd_py) / gp_ytd_py):.1%}"
        ,'mtd':  f"{gp_mtd:,.0f}"
        ,'mtd_py':  f"{gp_mtd_py:,.0f}"
        ,'ytd':  f"{gp_ytd:,.0f}"
        ,'ytd_py':  f"{gp_ytd_py:,.0f}"
    },
    {
        'name': 'GP %'
        ,'change': 0
        ,'mtd': f"{gp_margin_mtd:.1%}"
        ,'mtd_py': f"{gp_margin_mtd_py:.1%}"
        ,'ytd': f"{gp_margin_ytd:.1%}"
        ,'ytd_py': f"{gp_margin_ytd_py:.1%}"
    },
    {
        'name': 'TEUs'
        ,'change': f"{((teu_ytd - teu_ytd_py) / teu_ytd_py):.1%}"
        ,'mtd':  f"{teu_mtd:,.0f}"
        ,'mtd_py':  f"{teu_mtd_py:,.0f}"
        ,'ytd':  f"{teu_ytd:,.0f}"
        ,'ytd_py':  f"{teu_ytd_py:,.0f}"
    },
    {
        'name': 'Ch.weight'
        ,'change': f"{((chw_ytd - chw_ytd_py) / chw_ytd_py):.1%}"
        ,'mtd':  f"{chw_mtd:,.0f}"
        ,'mtd_py':  f"{chw_mtd_py:,.0f}"
        ,'ytd':  f"{chw_ytd:,.0f}"
        ,'ytd_py':  f"{chw_ytd_py:,.0f}"
    },
    {
        'name': 'Rev. (CL)'
        ,'change': f"{((rev_ytd_cl - rev_ytd_cl_py) / rev_ytd_cl_py):.1%}"
        ,'mtd':  f"{rev_mtd_cl:,.0f}"
        ,'mtd_py':  f"{rev_mtd_cl_py:,.0f}"
        ,'ytd':  f"{rev_ytd_cl:,.0f}"
        ,'ytd_py':  f"{rev_ytd_cl_py:,.0f}"
        ,'ytd_py_fm': f"{rev_ytd_py_fm:,.0f}"
        ,'mtd_py_fm': f"{rev_mtd_py_fm:,.0f}"
    },
    {
        'name': 'GP (CL)'
        ,'change': f"{((gp_ytd_cl - gp_ytd_cl_py) / gp_ytd_cl_py):.1%}"
        ,'mtd':  f"{gp_mtd_cl:,.0f}"
        ,'mtd_py':  f"{gp_mtd_cl_py:,.0f}"
        ,'ytd':  f"{gp_ytd_cl:,.0f}"
        ,'ytd_py':  f"{gp_ytd_cl_py:,.0f}"
        ,'ytd_py_fm': f"{gp_ytd_py_fm:,.0f}"
        ,'mtd_py_fm': f"{gp_mtd_py_fm:,.0f}"
    }
]


#### <mark>**COUNTRIES GP LIST**</mark>

In [31]:

def get_country_list(csv_str: str) -> list[dict]:

    keys = ['country', 'YOY %', 'YTD', 'YTD_PY', 'MTD', 'MTD_PY']
    return [dict(zip(keys, (row[0], row[6], row[1], row[5], row[3], row[4]))) for row in csv.reader(io.StringIO(csv_str.strip()))][1:]

countries_data = query.dax("""

                EVALUATE
            // _StartDateYTD_PY & _EndDateYTD_PY only for YTD_PY metric
                    VAR _RefDate = TODAY()
                    VAR _Day = DAY( _RefDate )
                    VAR _Month = MONTH( _RefDate )
                    VAR _Year = YEAR( _RefDate )

                    // Logic Conditions
                    VAR _IsJanEarly = _Month = 1 && _Day <= 10
                    VAR _IsJanLate = _Month = 1 && _Day > 10
                    // Note: Conditions 3 and 4 (Month > 1) are handled in the SWITCH default or specific branches below

                    // 1. Determine Start Date
                    // Logic: Only "Jan Early" goes back 2 years. All other scenarios start at Jan 1 of Prev Year.
                    VAR _StartDateYTD_PY = 
                        IF( 
                            _IsJanEarly, 
                            DATE( _Year - 2, 1, 1 ), 
                            DATE( _Year - 1, 1, 1 ) 
                        )

                    -- 2. Determine End Date
                    VAR _EndDateYTD_PY = 
                        SWITCH( TRUE(),
                            
                            // Cond 1: Jan Early => Last date of the current year set 2 years back (Dec 31, Year-2)
                            _IsJanEarly, DATE( _Year - 2, 12, 31 ),

                            // Cond 2: Jan Late => Last date of January set one year back (Jan 31, Year-1)
                            _IsJanLate, DATE( _Year - 1, 1, 31 ),

                            // Cond 3 & 4: Month > 1 => Last day of prev month set one year back
                            // (Logic is identical for Day <= 10 and Day > 10 based on your requirement)
                            // Calculates the last day of the previous month relative to the reference date, shifted 1 year back.
                            EDATE( EOMONTH( _RefDate, -1 ), -12 )
                        )
                    RETURN
						FILTER(
                            ADDCOLUMNS(
                                SUMMARIZECOLUMNS(
                                    'DAX Region Catalog'[Country Name]
                                                
                                            ,"__GP YTD"
                                                ,ROUND(
                                                    CALCULATE(
                                                        SUM('DAX'[AMOUNT_TAGETIK])
                                                            ,ALL('DAX'[ACCOUNTINGDATE])
                                                            ,'DAX'[ACCOUNTINGDATE] >= YTD_START
                                                            ,'DAX'[ACCOUNTINGDATE] <= YTD_END
                                                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                                                            )
                                                        ,0)
                                            ,"__GP YTD PY BASE"
                                                ,ROUND(
                                                    CALCULATE(
                                                        SUM('DAX'[AMOUNT_TAGETIK])
                                                            ,ALL('DAX'[ACCOUNTINGDATE])
                                                            ,'DAX'[ACCOUNTINGDATE] >= _StartDateYTD_PY
                                                            ,'DAX'[ACCOUNTINGDATE] <= _EndDateYTD_PY
                                                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                                                            )
                                                        ,0)
                                            ,"__GP MTD"
                                                ,ROUND(
                                                    CALCULATE(
                                                        SUM('DAX'[AMOUNT_TAGETIK])
                                                            ,ALL('DAX'[ACCOUNTINGDATE])
                                                            ,'DAX'[ACCOUNTINGDATE] >= MTD_START
                                                            ,'DAX'[ACCOUNTINGDATE] <= MTD_END
                                                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                                                            )
                                                        ,0)
                                            ,"__GP MTD PY"
                                                ,ROUND(
                                                    CALCULATE(
                                                        SUM('DAX'[AMOUNT_TAGETIK])
                                                            ,ALL('DAX'[ACCOUNTINGDATE])
                                                            ,'DAX'[ACCOUNTINGDATE] >= DATE(YEAR(TODAY())-1, MONTH(TODAY()),1)
                                                            ,'DAX'[ACCOUNTINGDATE] <= EOMONTH(DATE(YEAR(TODAY())-1, MONTH(TODAY()),1),0)
                                                            ,'DAX'[LINEOFBUSINESS] <> "CNLS"
                                                            )
                                                        ,0) / DAY(EOMONTH(DATE(YEAR(TODAY())-1, MONTH(TODAY()),1),0)) * ( DAY(TODAY())-1 )
                                                        
                                        )
								,"__GP YTD PY"
									,[__GP YTD PY BASE] + [__GP MTD PY]
                                ,"__YOY %"
                                    ,FORMAT((( [__GP YTD] - ([__GP YTD PY BASE] + [__GP MTD PY]) ) / ABS( COALESCE( ([__GP YTD PY BASE] + [__GP MTD PY]), [__GP YTD])) ),"percent")
                            ),NOT('DAX Region Catalog'[Country Name] in { "Côte d'Ivoire", "Sierra Leone"}))
                            ORDER BY [Country Name]
                        
""".replace('MTD_START', _filters['MTD_START']).replace('MTD_END', _filters['MTD_END'])
.replace('YTD_START', _filters['YTD_START']).replace('YTD_END', _filters['YTD_END'])
)

countries = get_country_list(countries_data)

#### <mark>**HTML TEMPLATE**</mark>

In [32]:


def KPI_table(metrics: list, countries: list, dates: dict, generation_date: str = None) -> str:
    """
    Generates a mobile-optimized, email-compatible HTML KPI dashboard.
    Accepts dynamic lists for main metrics and country breakdowns.
    """
    generation_date = generation_date or date.today().strftime("%Y-%m-%d")

    # --- 1. Strategic Helper Functions (Formatting) ---
    def format_val(val):
        """Intelligently formats currency and numbers (pos/neg), removing decimals."""
        if val is None: return ""

        # Native types handle negatives automatically in f-string formatting
        if isinstance(val, (int, float)): 
            return f"{int(round(val)):,}"

        val_str = str(val).strip()

        # Regex Amended: 
        # Added [\-\$]* at the start to allow negative signs and/or currency symbols
        if re.match(r'^[\-\$]*[\d,\$]+(\.\d+)?$', val_str):
            try:
                # Remove currency/commas, BUT keep the minus sign for float conversion
                clean = val_str.replace('$', '').replace(',', '')
                
                num = float(clean)
                formatted = f"{int(round(num)):,}"
                
                # If input had '$', prepend it to the result (e.g., "$-1,200")
                return f"${formatted}" if '$' in val_str else formatted
            except ValueError: 
                pass

        return val_str

    def format_pct(change_str):
        """Standardizes percentage display (e.g., '+13%' -> '+13.0%')."""
        if not change_str: return ""
        match = re.match(r'^([+-]?)(\d*\.?\d*)%?$', str(change_str).strip())
        if match:
            sign, num = match.groups()
            return f"{sign}{float(num):.1f}%" if num else change_str
        return change_str

    def get_color(val_str):
        """Returns semantic color hex based on string value (+/-)."""
        val = str(val_str).strip()
        return "#2e7d32" if val.startswith('+') else "#c62828" if val.startswith('-') else "#333"

    # --- 2. Row Rendering Closure (DRY Principle) ---
    def _render_row(name, yoy, ytd, ytd_py, mtd, mtd_py, ytd_fm="", mtd_fm="", is_last=False, is_header=False):
        border = "" if is_last else "border-bottom: 1px solid #e8e8e8;"
        bg_col_gray = "#C6D8D3" # The column highlight color
        bg_white = "#ffffff"
        
        # Determine text colors
        yoy_color = get_color(yoy) if not is_header else "#333"
        
        # Base styles
        td_base = "padding: 12px 10px; text-align: center; font-size: 14px;"
        td_name = f"padding: 12px 10px; font-weight: 500; font-size: 14px; color: #666; word-wrap: break-word; text-align: left;"
        
        return f"""
            <tr>
                <td style="{td_name} background: {bg_white}; {border}">{name}</td>
                <td style="{td_base} background: {bg_col_gray}; color: {yoy_color}; font-weight: 600; {border}">{yoy}</td>
                <td style="{td_base} background: {bg_col_gray}; color: #333; font-weight: 600; {border}">{ytd}</td>
                <td style="{td_base} background: {bg_col_gray}; color: #333; font-weight: 600; {border}">{ytd_py}</td>
                <td style="{td_base} background: {bg_white}; color: #333; {border}">{mtd}</td>
                <td style="{td_base} background: {bg_white}; color: #333; {border}">{mtd_py}</td>
                <td style="{td_base} background: {bg_white}; color: #333; {border}">{ytd_fm}</td>
                <td style="{td_base} background: {bg_white}; color: #333; {border}">{mtd_fm}</td>
            </tr>"""

    # --- 3. Build Table Rows ---
    table_rows_html = ""

    # A. Process Main Metrics
    for i, m in enumerate(metrics):
        # Insert "Contract Logistics" separator header if detected (specific business logic)
        if i > 0 and metrics[i-1]['name'] == 'Ch.weight':
             table_rows_html += """
        <tr><td colspan="8" style="background: #f5f7fa; padding: 12px 10px; font-weight: bold; font-size: 16px; color: #333; border-top: 2px solid #e8e8e8; border-bottom: 1px solid #e8e8e8;">
                            Contract logistics
                            <p style="margin: 4px 0 0; font-size: 10px; color: #888;">Without intercompany</p>
                            
        </td></tr>
             """

        # formatting
        yoy_val = format_pct(m.get('change', ''))
        # Hide YoY for Margin metrics if needed, or keep standard
        if "GP %" in m['name'] or "GP margin" in m['name']:
             yoy_display = "" 
        else:
             yoy_display = yoy_val

        table_rows_html += _render_row(
            name=m['name'],
            yoy=yoy_display,
            ytd=format_val(m.get('ytd')),
            ytd_py=format_val(m.get('ytd_py')),
            mtd=format_val(m.get('mtd')),
            mtd_py=format_val(m.get('mtd_py')),
            ytd_fm=format_val(m.get('ytd_py_fm', '')), # Dynamic lookup
            mtd_fm=format_val(m.get('mtd_py_fm', '')), # Dynamic lookup
            is_last=(i == len(metrics) - 1 and not countries) # Only remove border if no countries follow
        )

    # B. Process Countries (Dynamic Injection)
    if countries:
        # Section Header for Countries
        table_rows_html += """
        <tr><td colspan="8" style="background: #f5f7fa; padding: 12px 10px; font-weight: bold; font-size: 16px; color: #333; border-top: 2px solid #e8e8e8; border-bottom: 1px solid #e8e8e8;">
                            GP by country
                            <p style="margin: 4px 0 0; font-size: 10px; color: #888;">Without intercompany</p>
                            
        </td></tr>
        """
        
        for i, c in enumerate(countries):
            table_rows_html += _render_row(
                name=c.get('country', 'Unknown'),
                yoy=format_pct(c.get('YOY %', '')),
                ytd=format_val(c.get('YTD')),
                ytd_py=format_val(c.get('YTD_PY')),
                mtd=format_val(c.get('MTD')),
                mtd_py=format_val(c.get('MTD_PY')),
                ytd_fm="", # Countries don't have FM data in spec
                mtd_fm="", # Countries don't have FM data in spec
                is_last=(i == len(countries) - 1)
            )

    # --- 4. Footnotes (Data Driven) ---
    # Mapping logic to ensure safety against missing keys
    footnote_map = [
        ('¹', 'YTD dates', dates.get('YTD_START', '') + ' to ' + dates.get('YTD_END', '')),
        ('²', 'YTD PY dates', dates.get('YTD_START_PY', '') + ' to ' + dates.get('YTD_END_PY', '')),
        ('³', 'MTD dates', dates.get('MTD_START', '') + ' to ' + dates.get('MTD_END', '')),
        ('⁴', 'MTD PY dates', dates.get('MTD_START_PY', '') + ' to ' + dates.get('MTD_END_PY', '')),
        ('⁵', 'YTD PY FM dates', dates.get('YTD_START_PY_FM', '') + ' to ' + dates.get('YTD_END_PY_FM', '')),
        ('⁶', 'MTD PY FM dates', dates.get('MTD_START_PY_FM', '') + ' to ' + dates.get('MTD_END_PY_FM', '')),
    ]
    
    footer_html = "".join(f'<div style="margin-bottom: 2px;">{idx} {lbl}: {val}</div>' for idx, lbl, val in footnote_map)

    # --- 5. Final Assembly ---
    html_template = f"""
                <!DOCTYPE html>
                <html lang="en">
                <head>
                    <meta charset="UTF-8">
                    <meta name="viewport" content="width=device-width, initial-scale=1.0">
                    <title>Group KPI Dashboard</title>
                </head>
                <body style="margin: 0; padding: 16px 12px; font-family: Arial, Helvetica, sans-serif; background-color: #f5f7fa; color: #333;">
                    <table role="presentation" width="100%" style="font-family: Arial, Helvetica, sans-serif;">
                        <tr>
                            <td align="left">
                                <div style="margin-bottom: 16px;">
                                    <h2 style="margin: 0; font-size: 20px; font-weight: 800; color: #333;">ISS GF executive overview</h2>
                                    <p style="margin: 4px 0 0; font-size: 12px; color: #888;">Generation date: {generation_date}</p>
                                </div>
                                <table role="presentation" cellspacing="0" width="100%" style="border-collapse: collapse; background: #ffffff; font-size: 14px;">
                                    <tr>
                                        <th style="padding: 12px 10px; text-align: left; font-weight: 600; font-size: 12px; color: #666; border-bottom: 2px solid #e8e8e8; background: #ffffff;"> </th>
                                        <th style="padding: 12px 10px; text-align: center; font-weight: 600; font-size: 12px; color: #666; border-bottom: 2px solid #C6D8D3; background: #C6D8D3;">YoY %</th>
                                        <th style="padding: 12px 10px; text-align: center; font-weight: 600; font-size: 12px; color: #666; border-bottom: 2px solid #C6D8D3; background: #C6D8D3;">YTD<sup>¹</sup></th>
                                        <th style="padding: 12px 10px; text-align: center; font-weight: 600; font-size: 12px; color: #666; border-bottom: 2px solid #C6D8D3; background: #C6D8D3;">YTD PY<sup>²</sup></th>
                                        <th style="padding: 12px 10px; text-align: center; font-weight: 600; font-size: 12px; color: #666; border-bottom: 2px solid #e8e8e8; background: #ffffff;">MTD<sup>³</sup></th>
                                        <th style="padding: 12px 10px; text-align: center; font-weight: 600; font-size: 12px; color: #666; border-bottom: 2px solid #e8e8e8; background: #ffffff;">MTD PY<sup>⁴</sup></th>
                                        <th style="padding: 12px 10px; text-align: center; font-weight: 600; font-size: 12px; color: #666; border-bottom: 2px solid #e8e8e8; background: #ffffff;">YTD PY (FM)<sup>⁵</sup></th>
                                        <th style="padding: 12px 10px; text-align: center; font-weight: 600; font-size: 12px; color: #666; border-bottom: 2px solid #e8e8e8; background: #ffffff;">MTD PY (FM)<sup>⁶</sup></th>
                                    </tr>
                                    <tr><td colspan="8" style="background: #f5f7fa; padding: 12px 10px; font-weight: bold; font-size: 16px; color: #333; border-top: 2px solid #e8e8e8; border-bottom: 1px solid #e8e8e8;">
                                        
                                        Group indicators
                                        <p style="margin: 4px 0 0; font-size: 10px; color: #888;">Without intercompany / Without contract logistics</p>
                            
                                    </td></tr>

                                    {table_rows_html}
                                </table>
                                <div style="margin-top: 16px; font-size: 11px; color: #888; line-height: 1.4;">
                                    {footer_html}
                                </div>
                            </td>
                        </tr>
                    </table>
                </body>
                </html>
"""
    return html_template

#### <mark>**LOG METRICS**</mark>

In [33]:
json_str = json.dumps(metrics, ensure_ascii=False)
metrics_payload = json_str.replace("'", "''")

json_str = json.dumps(countries, ensure_ascii=False)
countries_payload = json_str.replace("'", "''")

In [34]:
%%tsql -artifact dwh
insert into kpi_email_log values 
(SYSUTCDATETIME(), N'{metrics_payload}', N'{countries_payload}',NULL)

#### <mark>**SEND EMAIL**</mark>

In [35]:
html = KPI_table(
    metrics = metrics
    ,dates = _dates
    ,countries = countries
)


if dt.now().weekday() ==  4: mailer.send_html(["kirill.bezzubkin@iss-gf.com"
                                                    ,'ivan.maksimov@iss-gf.com'
                                                    ,"lev.bondarenko@iss-gf.com"
                                                    ,'Enver.Moretti@iss-gf.com'
                                                    ,'Florian.Braun@iss-gf.com'
                                                    ,'Melvin.Chang@iss-gf.com'
                                                    ,'magesh.ganesan@iss-gf.com'
                                                    ]
                                                    ,html
                                                    ,subject='CEO daily report')